# Train NV

Import necessary libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

In [ ]:
print(f"Versión de TensorFlow: {tf.__version__}")

Versión de TensorFlow: 2.15.0


Download data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "..."

In [ ]:
X_train_vario = np.load(path + 'X_train_vario.npy')
y_train = np.load(path + 'y_train.npy') # (alpha, theta, ratio)

We standardize the target variables (parameters) to improve the performance of NN.

In [ ]:
y_scaler = StandardScaler()
y_train = y_scaler.fit_transform(y_train)

In [ ]:
y_scaler.mean_

array([1.49225652, 2.07506667, 0.49766667])

In [ ]:
y_scaler.scale_

array([0.90576534, 1.15612128, 0.2267739 ])

In [ ]:
N_vario = X_train_vario.shape[1]
N_vario

13

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(filters=128, kernel_size=8, input_shape=(N_vario,N_vario,1), activation='relu'),
    keras.layers.Conv2D(filters=128, kernel_size=4, activation='relu'),
    keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu'),
    keras.layers.Conv2D(filters=512, kernel_size=1, activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(3)
])
model.compile(optimizer=keras.optimizers.AdamW(),
              loss=keras.losses.MeanAbsoluteError(),
              metrics=[keras.metrics.RootMeanSquaredError()])
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_34 (Conv2D)          (None, 6, 6, 128)         8320      
                                                                 
 conv2d_35 (Conv2D)          (None, 3, 3, 128)         262272    
                                                                 
 conv2d_36 (Conv2D)          (None, 1, 1, 256)         295168    
                                                                 
 conv2d_37 (Conv2D)          (None, 1, 1, 512)         131584    
                                                                 
 flatten_9 (Flatten)         (None, 512)               0         
                                                                 
 dense_18 (Dense)            (None, 300)               153900    
                                                                 
 dense_19 (Dense)            (None, 3)                

In [ ]:
%%time
history = model.fit(X_train_vario, y_train,
                    epochs = 100,
                    batch_size = 500,
                    verbose = 1,
                    )

Epoch 1/100
900/900 [==============================] - 5s 5ms/step - loss: 0.2418 - root_mean_squared_error: 0.4580
Epoch 2/100
900/900 [==============================] - 5s 5ms/step - loss: 0.2408 - root_mean_squared_error: 0.4575
Epoch 3/100
900/900 [==============================] - 5s 6ms/step - loss: 0.2407 - root_mean_squared_error: 0.4568
Epoch 4/100
900/900 [==============================] - 5s 5ms/step - loss: 0.2406 - root_mean_squared_error: 0.4570
Epoch 5/100
900/900 [==============================] - 5s 5ms/step - loss: 0.2402 - root_mean_squared_error: 0.4565
Epoch 6/100
900/900 [==============================] - 5s 6ms/step - loss: 0.2401 - root_mean_squared_error: 0.4566
Epoch 7/100
900/900 [==============================] - 5s 5ms/step - loss: 0.2399 - root_mean_squared_error: 0.4562
Epoch 8/100
900/900 [==============================] - 5s 5ms/step - loss: 0.2396 - root_mean_squared_error: 0.4559
Epoch 9/100
900/900 [==============================] - 5s 6ms/step - los

In [ ]:
X_test_vario = np.load(path + 'X_test_vario.npy')

y_pred_test_vario = y_scaler.inverse_transform(model.predict(X_test_vario, verbose = 0))
y_pred_test_vario

array([[ 0.06645441,  1.0710648 ,  0.37913957],
       [-0.00959957,  1.1967041 ,  0.3742605 ],
       [-0.02616573,  1.0271382 ,  0.26993445],
       ...,
       [ 0.38786042,  2.9194703 ,  0.7120795 ],
       [ 0.75245845,  3.0528076 ,  0.8501949 ],
       [ 2.8722486 ,  2.6295319 ,  0.53547186]], dtype=float32)

In [ ]:
np.save(path+'y_pred_NV.npy', y_pred_test_vario)

Save trained NN

In [ ]:
model.save(path+'model_vario.keras')